In [22]:
# Importing libraries
import os
from dotenv import load_dotenv, find_dotenv
import base64
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Loading env variables
load_dotenv(find_dotenv())
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# Method to get spotify token
def getToken():
    authString  = f"{CLIENT_ID}:{CLIENT_SECRET}"
    authBytes   = authString.encode("utf-8")
    authBase64  = str(base64.b64encode(authBytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    tokenHeaders = {
        "Authorization" : f"Basic {authBase64}",
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    tokenData = {
        "grant_type" : "client_credentials"
    }
    tokenResponse = requests.post(
        url,
        headers=tokenHeaders,
        data=tokenData
    )
    tokenData   = tokenResponse.json()
    TOKEN       = tokenData['access_token']
    return TOKEN

# Method to get song ID
def getTrack(token, song_name, artist_name):
    url         = "https://api.spotify.com/v1/search"
    query       = f"?q=track:{song_name}%20artist:{artist_name}&type=track&limit=1"
    queryURL    = url+query
    songHeaders = {
        "Authorization" : f"Bearer {token}"
    }
    songResponse = requests.get(
        queryURL,
        headers=songHeaders
    )
    songData = songResponse.json()
    songDF   = pd.json_normalize(songData['tracks'], record_path='items')
    if len(songDF) > 0:
        ID = songDF['id'].iloc[0]
        songURI = songDF['uri'].iloc[0]
        songURI = songURI.split(':')[2]
        return ID, songURI
    else:
        return False

# Method to get song features
def getSongFeatures(token, songID):
    url = f'https://api.spotify.com/v1/audio-features?ids={songID}'
    songFeaturesHeaders = {
        "Authorization" : f"Bearer {token}"
    }
    featureResponse = requests.get(
        url,
        headers=songFeaturesHeaders
    )
    data    = featureResponse.json()
    dataDF  = pd.json_normalize(data['audio_features'])
    dataDF  = dataDF[['time_signature', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'loudness', 'tempo', 'duration_ms']]
    return dataDF

In [2]:
token1 = getToken()

In [23]:
t1 = getTrack(token1, 'Numb', 'Linkin Park')

In [24]:
d = getSongFeatures(token1, t1[0])

In [25]:
d

,time_signature,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,loudness,tempo,duration_ms
0,4,0.496,0.863,0.0381,0.0046,0,0.639,0.243,-4.153,110.018,185587


In [20]:
a = d.columns

In [21]:
a

Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature'],
      dtype='object')